In [3]:
import sqlite3
from sqlite3 import Error
import pandas as pd
import re

In [4]:
# create db
con = sqlite3.connect('FlightDatabase.db')

In [19]:
def sql_connection():
    try:
        con = sqlite3.connect('FlightDatabase.db')
        print('Connected to: FlightDatabase.db' )
        return con
    except Error:
        print(Error)
        
def create_table(con, create_sql):
    cursorObj = con.cursor()
    cursorObj.execute(create_sql)
    con.commit()            
    
def list_tables(con):
    cursorObj = con.cursor()
    cursorObj.execute('SELECT name from sqlite_master where type= "table"')
    print(cursorObj.fetchall())     

def sql_insert(con, insert_sql):
    cursorObj = con.cursor()    
    cursorObj.execute(insert_sql)
    con.commit()  
    
def table_schema(con, table):
    cursorObj = con.cursor()    
    print(sql_fetch(con, f"PRAGMA table_info('{table}');"))           
    
def sql_fetch(con, select_sql):
    cursorObj = con.cursor()
    cursorObj.execute(select_sql)
    rows = cursorObj.fetchall()
    for row in rows:
        print(row) 
        
def is_valid(row):
    col_is_3bigletter = [1,2,3,4]
    col_is_datetime = [5,6] 
    is_correct = True 
    #print(row)
    if len(row)== 7:         
        for i in col_is_3bigletter:
            if not re.match('^[A-Z]{3}$', row[i]):
                is_correct = False
        for i in col_is_datetime:
            if not re.match('^\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2}$', row[i]):
                is_correct = False    
    else:
        is_correct = False 
            
    return is_correct    

# 1.     Korzystając z biblioteki sqlite3 utwórz nową bazę danych z tabelą FlightLeg, która będzie zawierała informację o lotach samolotów, z następującymi kolumnami: 

·       id — identyfikator numeryczny nadawany z sekwencji 

·       tailNumber — identyfikator samolotu 

·       sourceAirportCode, destinationAirportCode — trzyliterowy kod lotniska (wg IATA) 

·       sourceCountryCode, destinationCountryCode — trzyliterowy kod kraju (wg ISO 3166-1 Alpha-3) 

·       departureTimeUtc, landingTimeUtc — data i czas (z dokładnością do sekundy) odpowiednio odlotu i lądowania samolotu (w UTC) 

In [13]:
create_sql ="""
CREATE TABLE if not exists FlightLeg(
id INTEGER PRIMARY KEY AUTOINCREMENT, 
tailNumber CHAR(10), 
sourceAirportCode CHAR(3), 
destinationAirportCode CHAR(3), 
sourceCountryCode CHAR(3), 
destinationCountryCode CHAR(3),
departureTimeUtc DATETIME,
landingTimeUtc DATETIME
)
"""
con = sql_connection()
create_table(con, create_sql)
con.close()

Connected to: FlightDatabase.db


In [27]:
con = sql_connection()
list_tables(con)
table_schema(con, 'FlightLeg')
con.close()

Connected to: FlightDatabase.db
[('FlightLeg',), ('sqlite_sequence',)]
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'tailNumber', 'CHAR(10)', 0, None, 0)
(2, 'sourceAirportCode', 'CHAR(3)', 0, None, 0)
(3, 'destinationAirportCode', 'CHAR(3)', 0, None, 0)
(4, 'sourceCountryCode', 'CHAR(3)', 0, None, 0)
(5, 'destinationCountryCode', 'CHAR(3)', 0, None, 0)
(6, 'departureTimeUtc', 'DATETIME', 0, None, 0)
(7, 'landingTimeUtc', 'DATETIME', 0, None, 0)
(8, 'flightDuration', 'integer', 0, None, 0)
(9, 'flightType', 'CHAR(1)', 0, None, 0)
None


In [14]:
con = sql_connection()
entities = f"'TEST', 'ACCa', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00'"
insert_sql = f"""
INSERT INTO FlightLeg VALUES(NULL,{entities})
"""
sql_insert(con, insert_sql)
con.close()

Connected to: FlightDatabase.db


In [35]:
con = sql_connection()
select_sql= """
SELECT * 
FROM FlightLeg
WHERE 1=1;
"""
sql_fetch(con, select_sql)
con.close()

Connected to: FlightDatabase.db
(1, 'TEST', 'ACCa', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', None, None)
(2, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00', None, None)
(3, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', None, None)
(4, 'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00', None, None)
(5, 'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00', None, None)
(6, 'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00', None, None)
(7, 'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00', None, None)
(8, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00', None, None)
(9, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00', None, None)
(10, 'A6-EUM', 'AMS', 'NLD', 'DXB', 'ARE', '2021-11-29 14:06:00', '20

# 2.     Wypełnij tabelę FlightLeg danymi z pliku: https://bitpeak.pl/datasets/flightlegs.csv 

In [ ]:
pd.read_csv('test.csv', delimiter=';')

In [20]:
df = pd.read_csv('flightlegs.csv', delimiter=';')
#df = pd.read_csv('test.csv', delimiter=';')
df = df.where(df.notnull(), 'None')
con = sql_connection()
for x in range(0,len(df.index)):    
    row = df.loc[x, :].values.tolist()
    if is_valid(row):
        entities = f"(NULL, {str(row)[1:-1]})"
        insert_sql = f"INSERT INTO FlightLeg VALUES{entities}"
        #print(insert_sql)
        sql_insert(con, insert_sql)  
con.close()

Connected to: FlightDatabase.db
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00')
INSERT INTO Flig

INSERT INTO FlightLeg VALUES(NULL, 'A6-ECX', 'BNE', 'AUS', 'DXB', 'ARE', '2021-11-30 11:11:30', '2021-12-01 00:59:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENJ', 'DAC', 'BGD', 'DXB', 'ARE', '2021-11-28 04:59:00', '2021-11-28 09:46:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGE', 'DXB', 'ARE', 'GRU', 'BRA', '2021-11-30 17:07:12', '2021-12-01 07:27:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUT', 'JFK', 'USA', 'DXB', 'ARE', '2021-11-29 16:12:00', '2021-11-30 04:03:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPE', 'DXB', 'ARE', 'ISB', 'PAK', '2021-11-29 23:20:00', '2021-11-30 02:00:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENK', 'BLQ', 'ITA', 'DXB', 'ARE', '2021-11-28 13:49:00', '2021-11-28 19:10:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENU', 'DXB', 'ARE', 'DEL', 'IND', '2021-11-30 18:08:41', '2021-11-30 20:53:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECH', 'DSS', 'SEN', 'DXB', 'ARE', '2021-11-28 17:57:19', '2021-11-29 02:56:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EQJ', 'DX

INSERT INTO FlightLeg VALUES(NULL, 'A6-EQA', 'SFO', 'USA', 'DXB', 'ARE', '2021-11-28 23:45:26', '2021-11-29 14:33:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUE', 'LHR', 'GBR', 'DXB', 'ARE', '2021-11-29 19:22:00', '2021-11-30 01:28:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECF', 'DXB', 'ARE', 'ALG', 'DZA', '2021-11-30 06:30:00', '2021-11-30 13:21:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EBU', 'BOM', 'IND', 'DXB', 'ARE', '2021-11-29 05:07:00', '2021-11-29 07:50:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPB', 'DXB', 'ARE', 'BOM', 'IND', '2021-11-29 17:47:40', '2021-11-29 20:28:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EQD', 'PER', 'AUS', 'DXB', 'ARE', '2021-11-30 14:16:56', '2021-12-01 00:47:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENH', 'CPH', 'DNK', 'DXB', 'ARE', '2021-11-28 13:23:10', '2021-11-28 19:19:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPQ', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-28 23:42:14', '2021-11-29 07:50:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUS', 'MR

INSERT INTO FlightLeg VALUES(NULL, 'A6-EOT', 'DXB', 'ARE', 'DME', 'RUS', '2021-11-30 12:32:00', '2021-11-30 17:26:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGM', 'LCK', 'USA', 'DXB', 'ARE', '2021-11-29 20:18:56', '2021-11-30 09:07:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENV', 'ACC', 'GHA', 'DXB', 'ARE', '2021-11-30 18:51:00', '2021-12-01 02:11:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EOC', 'CDG', 'FRA', 'DXB', 'ARE', '2021-11-30 13:52:00', '2021-11-30 19:53:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECM', 'RUH', 'SAU', 'DXB', 'ARE', '2021-11-29 01:45:33', '2021-11-29 03:20:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENI', 'DXB', 'ARE', 'PEW', 'PAK', '2021-11-29 23:38:54', '2021-11-30 02:02:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPV', 'DXB', 'ARE', 'ICN', 'KOR', '2021-11-30 23:56:19', '2021-12-01 07:38:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-END', 'DXB', 'ARE', 'LHE', 'PAK', '2021-11-30 18:19:41', '2021-11-30 20:47:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPN', 'IS

INSERT INTO FlightLeg VALUES(NULL, 'A6-EVO', 'DXB', 'ARE', 'LHR', 'GBR', '2021-11-28 11:08:00', '2021-11-28 18:37:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGG', 'SIN', 'SGP', 'PNH', 'KHM', '2021-11-28 02:17:07', '2021-11-28 03:50:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPS', 'DXB', 'ARE', 'DMM', 'SAU', '2021-11-29 12:18:00', '2021-11-29 13:22:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EWJ', 'BCN', 'ESP', 'DXB', 'ARE', '2021-11-29 14:14:00', '2021-11-29 20:16:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENO', 'ISB', 'PAK', 'DXB', 'ARE', '2021-11-29 23:13:05', '2021-11-30 02:21:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVP', 'LHR', 'GBR', 'DXB', 'ARE', '2021-11-29 14:03:00', '2021-11-29 20:29:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGB', 'DXB', 'ARE', 'CMN', 'MAR', '2021-11-29 05:20:00', '2021-11-29 13:28:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGG', 'CPH', 'DNK', 'LCK', 'USA', '2021-11-30 07:36:00', '2021-11-30 16:27:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECA', 'DX

...

INSERT INTO FlightLeg VALUES(NULL, 'A6-EGZ', 'DXB', 'ARE', 'BAH', 'BHR', '2021-11-27 12:38:00', '2021-11-27 13:30:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPS', 'IKA', 'IRN', 'DXB', 'ARE', '2021-11-30 00:41:42', '2021-11-30 02:34:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECY', 'SKT', 'PAK', 'DXB', 'ARE', '2021-11-30 05:24:00', '2021-11-30 08:47:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENR', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-28 17:23:00', '2021-11-28 20:45:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGZ', 'DXB', 'ARE', 'KWI', 'KWT', '2021-11-30 17:09:00', '2021-11-30 18:41:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPB', 'AMM', 'JOR', 'DXB', 'ARE', '2021-11-30 23:20:40', '2021-12-01 02:03:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECO', 'BLR', 'IND', 'DXB', 'ARE', '2021-11-30 23:31:00', '2021-12-01 03:05:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EWC', 'DXB', 'ARE', 'BAH', 'BHR', '2021-11-29 12:42:00', '2021-11-29 13:35:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECD', 'DX

INSERT INTO FlightLeg VALUES(NULL, 'A6-EPB', 'DXB', 'ARE', 'ADD', 'ETH', '2021-11-30 06:18:00', '2021-11-30 09:59:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECQ', 'KUL', 'MYS', 'DXB', 'ARE', '2021-11-29 17:31:50', '2021-11-30 00:03:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVH', 'DXB', 'ARE', 'IAD', 'USA', '2021-11-29 22:19:00', '2021-11-30 13:04:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPL', 'DXB', 'ARE', 'LAD', 'AGO', '2021-11-28 05:58:00', '2021-11-28 13:21:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVQ', 'LHR', 'GBR', 'DXB', 'ARE', '2021-11-30 13:44:00', '2021-11-30 20:17:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUM', 'DXB', 'ARE', 'AMS', 'NLD', '2021-11-30 04:30:00', '2021-11-30 11:29:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPA', 'DXB', 'ARE', 'CRK', 'PHL', '2021-11-27 23:01:00', '2021-11-28 07:42:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECR', 'MIA', 'USA', 'DXB', 'ARE', '2021-11-30 02:02:54', '2021-11-30 15:42:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECZ', 'DX

INSERT INTO FlightLeg VALUES(NULL, 'A6-EGY', 'DXB', 'ARE', 'MAA', 'IND', '2021-11-29 23:15:00', '2021-11-30 02:51:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENY', 'LCA', 'CYP', 'DXB', 'ARE', '2021-11-28 17:20:36', '2021-11-28 20:47:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENK', 'DXB', 'ARE', 'MLE', 'MDV', '2021-11-29 22:57:01', '2021-11-30 02:43:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVI', 'CAI', 'EGY', 'DXB', 'ARE', '2021-11-29 10:45:00', '2021-11-29 13:34:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EQM', 'DXB', 'ARE', 'KIX', 'JPN', '2021-11-29 23:25:00', '2021-11-30 07:56:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGP', 'LCK', 'USA', 'FRA', 'DEU', '2021-11-30 21:16:23', '2021-12-01 04:54:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENH', 'KHI', 'PAK', 'DXB', 'ARE', '2021-11-30 07:14:07', '2021-11-30 09:10:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENE', 'MLE', 'MDV', 'DXB', 'ARE', '2021-11-28 17:50:49', '2021-11-28 21:43:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVC', 'MR

INSERT INTO FlightLeg VALUES(NULL, 'A6-ECM', 'DXB', 'ARE', 'RUH', 'SAU', '2021-11-28 21:58:00', '2021-11-28 23:42:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENO', 'SKT', 'PAK', 'DXB', 'ARE', '2021-11-28 21:27:00', '2021-11-29 00:50:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECH', 'CRK', 'PHL', 'DXB', 'ARE', '2021-11-30 12:12:13', '2021-11-30 20:30:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUW', 'DXB', 'ARE', 'GRU', 'BRA', '2021-11-30 06:05:00', '2021-11-30 20:14:37')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EQF', 'DXB', 'ARE', 'BRU', 'BEL', '2021-11-28 10:11:00', '2021-11-28 17:25:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EVC', 'DXB', 'ARE', 'MRU', 'MUS', '2021-11-29 22:45:00', '2021-11-30 04:52:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGZ', 'KWI', 'KWT', 'DXB', 'ARE', '2021-11-30 19:53:55', '2021-11-30 21:22:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENO', 'DXB', 'ARE', 'SKT', 'PAK', '2021-11-28 17:18:44', '2021-11-28 19:54:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ENE', 'DX

INSERT INTO FlightLeg VALUES(NULL, 'A6-ECF', 'DXB', 'ARE', 'IKA', 'IRN', '2021-11-28 04:04:00', '2021-11-28 05:51:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EQI', 'DXB', 'ARE', 'KWI', 'KWT', '2021-11-29 21:42:00', '2021-11-29 23:14:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGZ', 'DXB', 'ARE', 'MED', 'SAU', '2021-11-29 10:47:41', '2021-11-29 13:07:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGV', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-28 23:14:37', '2021-11-29 03:16:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EGF', 'DXB', 'ARE', 'PRG', 'CZE', '2021-11-30 05:33:00', '2021-11-30 11:48:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-ECG', 'KUL', 'MYS', 'AKL', 'NZL', '2021-11-29 13:03:00', '2021-11-29 22:30:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EBU', 'MXP', 'ITA', 'DXB', 'ARE', '2021-11-28 13:55:38', '2021-11-28 19:28:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EPB', 'DXB', 'ARE', 'BAH', 'BHR', '2021-11-28 05:33:00', '2021-11-28 06:26:00')
INSERT INTO FlightLeg VALUES(NULL, 'A6-EUC', 'MU

In [33]:
con = sql_connection()
select_sql= """
SELECT 
*
--count(*) 
FROM FlightLeg
WHERE 1=1;
"""
sql_fetch(con, select_sql)
con.close()

Connected to: FlightDatabase.db
(1, 'TEST', 'ACCa', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', None, None)
(2, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00', None, None)
(3, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', None, None)
(4, 'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00', None, None)
(5, 'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00', None, None)
(6, 'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00', None, None)
(7, 'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00', None, None)
(8, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00', None, None)
(9, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00', None, None)
(10, 'A6-EUM', 'AMS', 'NLD', 'DXB', 'ARE', '2021-11-29 14:06:00', '20

# 3.     Dodaj do tabeli dwie nowe kolumny: 

·       flightDuration — wypełnij ją wartościami określającymi czas trwania lotu w minutach (zaokrągl wartości do najbliższej liczby całkowitej) 

In [25]:
con = sql_connection()
select_sql="ALTER TABLE FlightLeg ADD COLUMN flightDuration INTEGER"
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
None


In [ ]:
con = sql_connection()
sql= """
UPDATE FlightLeg
SET flightDuration = Cast (( JulianDay(landingTimeUtc) - JulianDay(departureTimeUtc)) * 24 * 60 As Integer)
"""
cursorObj = con.cursor()
cursorObj.execute(sql)
con.commit()
print(sql_fetch(con, select_sql))
con.close()

·       flightType — wypełnij ją wartościami określającymi typ lotu: krajowy (wartość ‘D’ = domestic) lub zagraniczny (wartość ‘I’ = international); lot krajowy to lot zaczynający się i kończący w tym samym kraju 

In [26]:
con = sql_connection()
select_sql="ALTER TABLE FlightLeg ADD COLUMN flightType CHAR(1)"
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
None


In [39]:
con = sql_connection()
sql= """
UPDATE FlightLeg
SET flightType = Case when sourceCountryCode == destinationCountryCode then 'D' else 'I' END
"""
cursorObj = con.cursor()
cursorObj.execute(sql)
con.commit()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
(1, 'TEST', 'ACCa', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', 42, 'I')
(2, 'A6-EUC', 'DXB', 'ARE', 'CAI', 'EGY', '2021-11-30 04:47:00', '2021-11-30 08:25:00', 217, 'I')
(3, 'A6-ENE', 'ACC', 'GHA', 'ABJ', 'CIV', '2021-11-29 13:35:00', '2021-11-29 14:17:00', 42, 'I')
(4, 'A6-END', 'DXB', 'ARE', 'MNL', 'PHL', '2021-11-27 23:50:11', '2021-11-28 07:58:00', 487, 'I')
(5, 'A6-EPD', 'IST', 'TUR', 'DXB', 'ARE', '2021-11-28 17:51:53', '2021-11-28 21:31:00', 219, 'I')
(6, 'A6-EUU', 'JED', 'SAU', 'DXB', 'ARE', '2021-11-28 08:53:00', '2021-11-28 11:10:00', 137, 'I')
(7, 'A6-EPF', 'DXB', 'ARE', 'TUN', 'TUN', '2021-11-28 05:30:00', '2021-11-28 11:58:00', 388, 'D')
(8, 'A6-EVF', 'DXB', 'ARE', 'JFK', 'USA', '2021-11-30 22:57:00', '2021-12-01 13:01:00', 843, 'I')
(9, 'A6-ENS', 'DXB', 'ARE', 'SEZ', 'SYC', '2021-11-30 22:39:38', '2021-12-01 02:41:00', 241, 'I')
(10, 'A6-EUM', 'AMS', 'NLD', 'DXB', 'ARE', '2021-11-29 14:06:00', '2021-11-29 19:51:00', 

# 4.     Zaimplementuj logikę odpowiadającą na poniższe pytania: 

    5.     (punkt opcjonalny) Odstęp pomiędzy którymi lotami tego samego dowolnego samolotu był najkrótszy i ile minut trwał? 

    1.     Który samolot wykonał najwięcej lotów? 

In [44]:
select_sql= """
SELECT tailNumber, count(tailNumber) as flight_cnt
FROM FlightLeg
WHERE 1=1
group by tailNumber
ORDER BY flight_cnt DESC LIMIT 1;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
('A6-ENS', 26)
None


    2.     Który samolot przeleciał najwięcej minut? 

In [45]:
select_sql= """
SELECT tailNumber, SUM(flightDuration) AS sum_duration
FROM FlightLeg
WHERE 1=1
group by tailNumber
ORDER BY sum_duration DESC LIMIT 1;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
('A6-EVF', 6844)
None


    3.     Który lot, w podziale na krajowe i zagraniczne, był najkrótszy, a który najdłuższy, i ile minut trwał? 

In [79]:
select_sql= """
SELECT 
id,
flightType,
flightDuration,
MAX(flightDuration)as MAX,
MIN(flightDuration)as MIN
FROM FlightLeg
where flightType ='I'
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
(573, 'I', 41, 966, 41)
None


In [76]:
select_sql= """
SELECT * FROM (
SELECT 
id,
flightType,
flightDuration,
'MAX' as type,
ROW_NUMBER() OVER(PARTITION BY flightType,flightType ORDER BY flightDuration DESC) rowNumber
FROM FlightLeg
) tmax
where tmax.rowNumber = 1

UNION

SELECT tmin.* FROM (
SELECT 
id,
flightType,
flightDuration,
'MIN' as type,
ROW_NUMBER() OVER(PARTITION BY flightType,flightType ORDER BY flightDuration ASC) rowNumber
FROM FlightLeg
)tmin
where tmin.rowNumber = 1
;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
(21, 'D', 738, 'MAX', 1)
(275, 'D', 58, 'MIN', 1)
(284, 'I', 966, 'MAX', 1)
(573, 'I', 41, 'MIN', 1)
None


    4.     (punkt opcjonalny) Ile jest błędnych rekordów opisujących loty, które wskazują, że samolot wykonywał więcej niż jeden lot jednocześnie? Wyświetl wszystkie pary takich konfliktowych lotów.


In [ ]:
select_sql= """
SELECT tmp.* FROM (
SELECT 
id,
flightType,
flightDuration,
ROW_NUMBER() OVER(PARTITION BY flightType,flightType ORDER BY flightDuration DESC) rowNumber
FROM FlightLeg
)tmp
where tmp.rowNumber =1;
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

In [46]:
select_sql= """PRAGMA table_info('FlightLeg');
"""
con = sql_connection()
print(sql_fetch(con, select_sql))
con.close()

Connected to: FlightDatabase.db
(0, 'id', 'INTEGER', 0, None, 1)
(1, 'tailNumber', 'CHAR(10)', 0, None, 0)
(2, 'sourceAirportCode', 'CHAR(3)', 0, None, 0)
(3, 'destinationAirportCode', 'CHAR(3)', 0, None, 0)
(4, 'sourceCountryCode', 'CHAR(3)', 0, None, 0)
(5, 'destinationCountryCode', 'CHAR(3)', 0, None, 0)
(6, 'departureTimeUtc', 'DATETIME', 0, None, 0)
(7, 'landingTimeUtc', 'DATETIME', 0, None, 0)
(8, 'flightDuration', 'integer', 0, None, 0)
(9, 'flightType', 'CHAR(1)', 0, None, 0)
None


In [ ]:
"alter table FlightLeg2 add column 'flightDuration' 'float'"

In [ ]:
cursor = con.cursor()
data=cursor.execute('''SELECT * FROM FlightLeg2''')
for column in data.description:
    print(column[0])
      

In [ ]:
print(len(df.index))

In [ ]:
con = sql_connection('FlightDatabase.db')
for x in range(0,len(df.index)):
    row = df.loc[x, :].values.tolist()
    
    entities = f"{str(row)[1:-1]}"
    insert_sql = f"""
    INSERT INTO FlightLeg2 VALUES(NULL,{entities})
    """
    print(insert_sql)
    
    sql_insert(con, insert_sql)
    

In [ ]:
select_sql= """
SELECT *,
 
Cast (( JulianDay(landingTimeUtc) - JulianDay(departureTimeUtc)) * 24 * 60 As Integer) as flightDuration 
FROM FlightLeg2
WHERE id = 2
"""
print(sql_fetch(con, select_sql))